In [1]:
import pandas as pd
import cx_Oracle


from sqlalchemy import create_engine
import xlsxwriter

In [2]:
db_config = {
    'user': 'user', # имя пользователя
    'pwd': '1111111', # пароль
    'host': 'двх',
    'port': 1521, # порт подключения
    'db': 'двх' # название базы данных
}  

connection_string = 'oracle://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

In [3]:
engine = create_engine(connection_string) 

In [4]:
query = '''
Select 'Продажи' "Тип данных",
       dc.nm "Аптека",
       dma.cd "Код аналога",
       dmn.cd "Код ЛС",
       dmn.cd_goods "Код ЕФ",
       dmn.nm "Наименование ЛС",
       dmk.nm "Вид ЛС",
       omn.level_1 "ФГ1",
       omn.level_2 "ФГ2",
       dmn.pz_recipe "Вид отпуска",
       dmn.pz_jnivls "ЖНВЛП", 
       omn.mar_category "Маркетинговая категория",
       (sum(f.qty * k.rest_kind * -1) / 6) "Среднее"
  from dwh.fact_move_goods f
  left join dwh.dim_operation_kind k
    on f.operation_kind_key = k.operation_kind_key
  left join dwh.dim_contragent dc
    on f.owner_key = dc.contragent_key
  left join dwh.dim_department dd
    on f.department_key = dd.department_key
  left join dwh.dim_med_nomenclature dmn
    on f.med_nomenclature_key = dmn.med_nomenclature_key
  left join dwh.dim_med_kind dmk
    on dmn.med_kind_key = dmk.med_kind_key
  left join dwh.dim_med_analog dma
    on f.med_analog_key = dma.med_analog_key
  left join dwh.mv_olap_med_nomenclature omn
    on f.med_nomenclature_key = omn.med_nomenclature_key
  left join dwh.dim_department_type ddt
    on dd.department_type_key = ddt.department_type_key
 where f.account_type_key = 2
   and f.operation_kind_key in (2, 6)
   and f.data_type = 12
   and f.year_at in (2021)
   and f.cheque_key is not null
   and (EXTRACT(MONTH FROM f.dt_operation) IN(2,3,6,7,8,9))
   AND ddt.cd IN (10,  16,  20, 23)
-- 10 = розница, 20 = ветеринария, 16 = ортопедия, 15 = оптика материалы, 19 = оптика производство, 23 = оптика.
   group by dc.cd,
       dc.nm,       
       dma.cd,
       dma.nm,
       dmn.cd,
       dmn.cd_goods,
       dmn.nm,
       dmk.nm,
       omn.level_1,
       omn.level_2,
       dmn.pz_recipe,
       dmn.pz_jnivls,
       omn.mar_category    
'''

prodazhi = pd.read_sql_query(query, con=engine)

In [6]:
kategorii = pd.read_excel('Данные для отчета.xlsx')
kategorii.columns = ['Аптека', 'Категория']

In [18]:
prodazhi = pd.read_csv('prodazhi.csv', index_col=0)

In [7]:
kategorii['Категория'] = kategorii['Категория'].replace('а/п ЛПУвысокий город', 'ап ЛПУвысокий город')
kategorii['Категория'] = kategorii['Категория'].replace('а/п ЛПУ край', 'ап ЛПУ край')
kategorii['Категория'] = kategorii['Категория'].replace('Льгота+', 'Льгота')
kategorii['Категория'] = kategorii['Категория'].str.lower()

In [19]:
prodazhi = prodazhi.merge(kategorii, on='Аптека')

Предобработка продаж

In [20]:
def predobrabotka(local_df):
    local_df['Вид отпуска'] = local_df['Вид отпуска'].fillna('OTC').replace(1, 'RX')
    local_df['ЖНВЛП'] = local_df['ЖНВЛП'].replace(1, '+')
    local_df = local_df.rename({'Вид отпуска':'Rx/OTC'}, axis=1)
    return local_df

In [21]:
prodazhi = predobrabotka(prodazhi)


In [21]:
query = '''
Select 'Продажи' "Тип данных",
       dc.nm "Аптека",
       dma.cd "Код аналога",
       dmn.cd "Код ЛС",
       dmn.cd_goods "Код ЕФ",
       dmn.nm "Наименование ЛС",
       dmk.nm "Вид ЛС",
       omn.level_1 "ФГ1",
       omn.level_2 "ФГ2",
       dmn.pz_recipe "Вид отпуска",
       dmn.pz_jnivls "ЖНВЛП", 
       omn.mar_category "Маркетинговая категория",
       (sum(f.qty * k.rest_kind * -1) / 6) "Среднее"
  from dwh.fact_move_goods f
  left join dwh.dim_operation_kind k
    on f.operation_kind_key = k.operation_kind_key
  left join dwh.dim_contragent dc
    on f.owner_key = dc.contragent_key
  left join dwh.dim_department dd
    on f.department_key = dd.department_key
  left join dwh.dim_med_nomenclature dmn
    on f.med_nomenclature_key = dmn.med_nomenclature_key
  left join dwh.dim_med_kind dmk
    on dmn.med_kind_key = dmk.med_kind_key
  left join dwh.dim_med_analog dma
    on f.med_analog_key = dma.med_analog_key
  left join dwh.mv_olap_med_nomenclature omn
    on f.med_nomenclature_key = omn.med_nomenclature_key
  left join dwh.dim_department_type ddt
    on dd.department_type_key = ddt.department_type_key
 where f.account_type_key = 2
   and f.operation_kind_key in (2, 6)
   and f.data_type = 12
   and f.year_at in (2021)
   and f.cheque_key is not null
   and (EXTRACT(MONTH FROM f.dt_operation) IN(2,3,6,7,8,9))
   AND ddt.cd IN (24)
-- 10 = розница, 20 = ветеринария, 16 = ортопедия, 15 = оптика материалы, 19 = оптика производство, 23 = оптика.
   group by dc.cd,
       dc.nm,       
       dma.cd,
       dma.nm,
       dmn.cd,
       dmn.cd_goods,
       dmn.nm,
       dmk.nm,
       omn.level_1,
       omn.level_2,
       dmn.pz_recipe,
       dmn.pz_jnivls,
       omn.mar_category
'''

farmacia = pd.read_sql_query(query, con=engine)

In [22]:
farmacia = farmacia.merge(kategorii, on='Аптека', how='inner')

In [23]:
farmacia = predobrabotka(farmacia)


In [25]:
query = '''
Select 'Продажи' "Тип данных",
       dc.nm "Аптека",
       dma.cd "Код аналога",
       dmn.cd "Код ЛС",
       dmn.cd_goods "Код ЕФ",
       dmn.nm "Наименование ЛС",
       dmk.nm "Вид ЛС",
       omn.level_1 "ФГ1",
       omn.level_2 "ФГ2",
       dmn.pz_recipe "Вид отпуска",
       dmn.pz_jnivls "ЖНВЛП", 
       omn.mar_category "Маркетинговая категория",
       (sum(f.qty * k.rest_kind * -1) / 6) "Среднее"
  from dwh.fact_move_goods f
  left join dwh.dim_operation_kind k
    on f.operation_kind_key = k.operation_kind_key
  left join dwh.dim_contragent dc
    on f.owner_key = dc.contragent_key
  left join dwh.dim_department dd
    on f.department_key = dd.department_key
  left join dwh.dim_med_nomenclature dmn
    on f.med_nomenclature_key = dmn.med_nomenclature_key
  left join dwh.dim_med_kind dmk
    on dmn.med_kind_key = dmk.med_kind_key
  left join dwh.dim_med_analog dma
    on f.med_analog_key = dma.med_analog_key
  left join dwh.mv_olap_med_nomenclature omn
    on f.med_nomenclature_key = omn.med_nomenclature_key
  left join dwh.dim_department_type ddt
    on dd.department_type_key = ddt.department_type_key
 where f.account_type_key = 2
   and f.operation_kind_key in (2, 6)
   and f.data_type = 12
   and f.year_at in (2021)
   and f.cheque_key is not null
   and (EXTRACT(MONTH FROM f.dt_operation) IN(2,3,6,7,8,9))
   AND ddt.cd IN (17)
-- 10 = розница, 20 = ветеринария, 16 = ортопедия, 15 = оптика материалы, 19 = оптика производство, 23 = оптика.
   group by dc.cd,
       dc.nm,       
       dma.cd,
       dma.nm,
       dmn.cd,
       dmn.cd_goods,
       dmn.nm,
       dmk.nm,
       omn.level_1,
       omn.level_2,
       dmn.pz_recipe,
       dmn.pz_jnivls,
       omn.mar_category
'''

aptekaru = pd.read_sql_query(query, con=engine)

In [26]:
aptekaru = aptekaru.merge(kategorii, on='Аптека', how='inner')

In [27]:
aptekaru = predobrabotka(aptekaru)


In [28]:
d3d4 = pd.read_excel('Данные для отчета.xlsx', 1, usecols=['Наименование аптеки', 'Группа товара', 'Код ГПКК']).rename({'Код ГПКК':'Код ЛС', 'Наименование аптеки':'Аптека'}, axis=1)

In [29]:
d3d4 = d3d4.loc[d3d4['Код ЛС'] > 0]

In [30]:
d3d4 = d3d4.merge(kategorii, on='Аптека', how='inner')

In [39]:
query = '''
select 
       dc.nm "Аптека",
       dma.cd "Код аналога",
       dmn.cd "Код ЛС",
       dmn.cd_goods "Код ЕФ",
       dmn.nm "Наименование ЛС",
       dmk.nm "Вид ЛС",
       omn.level_1 "ФГ1",
       omn.level_2 "ФГ2",
       dmn.pz_recipe "Вид отпуска",
       dmn.pz_jnivls "ЖНВЛП", 
       omn.mar_category "Маркетинговая категория",
       sum(b.qty) "Количество"
  from dwh.xml_balance_goods_ef2 b
  left join dwh.dim_contragent dc
    on b.owner_key = dc.contragent_key
  left join dwh.dim_med_nomenclature dmn
    on b.med_nomenclature_key = dmn.med_nomenclature_key
  left join dwh.dim_med_analog dma
    on dmn.med_analog_key = dma.med_analog_key
  left join dwh.dim_department dd
    on b.department_key = dd.department_key
  left join dwh.dim_department_type ddt
    on dd.department_type_key = ddt.department_type_key
  left join dwh.dim_med_kind dmk
    on dmn.med_kind_key = dmk.med_kind_key
  left join dwh.mv_olap_med_nomenclature omn
    on dmn.med_nomenclature_key = omn.med_nomenclature_key
 where b.dt_rest = '26.02.2022'
    AND ddt.cd IN (10, 16, 20, 23)
-- 10 = розница, 20 = ветеринария, 16 = ортопедия, 15 = оптика материалы, 19 = оптика производство, 23 = оптика.
 group by  
    dc.cd,
       dc.nm,       
       dma.cd,
       dma.nm,
       dmn.cd,
       dmn.cd_goods,
       dmn.nm,
       dmk.nm,
       omn.level_1,
       omn.level_2,
       dmn.pz_recipe,
       dmn.pz_jnivls,
       omn.mar_category    
'''

ostatki = pd.read_sql_query(query, con=engine)

In [ ]:
ostatki = ostatki.merge(kategorii, on='Аптека', how='inner')

In [40]:
ostatki = predobrabotka(ostatki)

In [41]:
stop_list = pd.read_excel('Данные для отчета.xlsx', 2)
stop_list = predobrabotka(stop_list)
arrays = [['код', 'х-ки'], stop_list.columns]
stop_list.columns = pd.MultiIndex.from_arrays(arrays)

In [46]:
abc = pd.read_csv('abc.csv', index_col=0)


In [47]:
abc = abc[['Наименование аптеки',	'Код ГПКК',	'Группа товара']]

In [48]:
abc.columns = ['Аптека', 'Код ЛС', 'Группа товара']
abc = abc.merge(kategorii, on='Аптека', how='inner')
abc = abc.dropna()
abc['Категория'] = abc['Категория'].str.lower()
abc_list = ['CAY', 'BBY', 'BBX', 'BAY', 'BAX', 'ACY', 'ACX', 'ABY', 'ABX', 'AAZ', 'AAY', 'AAX']

In [49]:
def get_excel(pivot, cat_name, first_row, second_row):
    writer = pd.ExcelWriter(cat_name + '.xlsx', engine='xlsxwriter')

    workbook = writer.book
    def get_header(name):
        color = {'код': '#d3d4f5',
                'наименование': '#f5d3d3',
                'х-ки': '#f5e7d3',
                'Зал': '#ecf5d3',
                'Д3Д4': '#d3f5d7',
                'Апека.ру': '#d3f5e7',
                '24фармацея': '#d3ebf5',
                'Остатки': '#d3dcf5',
                'ABC': '#c97d7d'}

        return workbook.add_format({'bold':True,
                                            'text_wrap':True,
                                            'valign':'top',
                                            'align':'center',
                                            'fg_color':color[name],
                                            'border':2,
                                            'font_size':9})
        

    table_format = workbook.add_format({'border': 1,
                                        'font_size':9,
                                        'num_format':'0.00'})
    cod_format = workbook.add_format({'border': 1,
                                        'font_size':9})
    percent_format = workbook.add_format({'border': 1,
                                        'font_size':9,
                                        'num_format':'0.0%'})
    conditional_format = workbook.add_format({'border':1,
                                            'font_size':9,
                                            'bg_color':'#FFC7CE',
                                            'font_color':'#9C0006'})


    pivot.to_excel(writer, sheet_name='Категория', startrow=1, header=False)

    worksheet = writer.sheets['Категория']

    for name, (max_col, min_col)  in first_row.items():
        
        header_format = get_header(name)
        if name != 'наименование':
            worksheet.merge_range(0,min_col +1 ,0 , max_col + 1,name, header_format)
            for i in range(min_col+1, max_col + 2):
                worksheet.write(1, i , second_row[i-1], header_format)
                
        else:
            worksheet.write(0, min_col +1, name, header_format)
            worksheet.write(1, min_col +1, second_row[min_col], header_format)

    for g in abc_list:
        worksheet.conditional_format(2, first_row['ABC'][1]+1, pivot.shape[0] +1, first_row['ABC'][0]-1, {'type':     'cell',
                                                    'criteria': '==',
                                                    'value':    f'"{g}"',
                                                    'format':   conditional_format})

    for row_for_height in range(1, pivot.shape[0]+2):
        worksheet.set_row(row_for_height, 12)

    worksheet.set_row(1, 132)

    worksheet.autofilter(1, 1, pivot.shape[1], pivot.shape[1]-1)
    worksheet.set_column(first_row['код'][1]+1, first_row['код'][0], 7, cod_format, {'level': 1, 'hidden': True})
    worksheet.set_column(first_row['код'][0]+1, first_row['код'][0]+1, 7, cod_format)
    worksheet.set_column(first_row['наименование'][1]+1, first_row['наименование'][1]+1, 24, cod_format)

    worksheet.set_column(5, 6 , 24,table_format, {'level': 1, 'hidden': True})
    worksheet.set_column(7, 7, 12, table_format, {'level': 1, 'hidden': True})
    worksheet.set_column(8, 9, 7, table_format, {'level': 1, 'hidden': True})
    worksheet.set_column(10, 10, 16, table_format, {'level': 1, 'hidden': True})
    worksheet.set_column(11, 11, 7, table_format)

    worksheet.set_column(first_row['ABC'][1]+1, first_row['ABC'][0] - 1, 7, table_format, {'level': 1, 'hidden': True})
    worksheet.set_column(first_row['ABC'][0], first_row['ABC'][0], 7, table_format)
    worksheet.set_column(first_row['ABC'][0]+1, first_row['ABC'][0]+1, 7, percent_format)

    worksheet.set_column(first_row['Зал'][1]+1, first_row['Зал'][0] - 2, 7, table_format, {'level': 1, 'hidden': True})
    worksheet.set_column(first_row['Зал'][0]-1, first_row['Зал'][0]-1, 7, table_format)
    worksheet.set_column(first_row['Зал'][0], first_row['Зал'][0], 7, percent_format)
    worksheet.set_column(first_row['Зал'][0]+1, first_row['Зал'][0]+1, 7, table_format)

    worksheet.set_column(first_row['Д3Д4'][1]+1, first_row['Д3Д4'][0] - 3, 7, table_format, {'level': 1, 'hidden': True})
    worksheet.set_column(first_row['Д3Д4'][0]-2, first_row['Д3Д4'][0]-2, 7, table_format)
    worksheet.set_column(first_row['Д3Д4'][0]-1, first_row['Д3Д4'][0]-1, 7, percent_format)
    worksheet.set_column(first_row['Д3Д4'][0], first_row['Д3Д4'][0], 7, table_format)
    worksheet.set_column(first_row['Д3Д4'][0]+1, first_row['Д3Д4'][0]+1, 7, percent_format)

    worksheet.set_column(first_row['Апека.ру'][1]+1, first_row['Апека.ру'][0] - 2, 7, table_format, {'level': 1, 'hidden': True})
    worksheet.set_column(first_row['Апека.ру'][0]-1, first_row['Апека.ру'][0]-1, 7, table_format)
    worksheet.set_column(first_row['Апека.ру'][0], first_row['Апека.ру'][0], 7, percent_format)
    worksheet.set_column(first_row['Апека.ру'][0]+1, first_row['Апека.ру'][0]+1, 7, table_format)

    worksheet.set_column(first_row['24фармацея'][1]+1, first_row['24фармацея'][0] - 2, 7, table_format, {'level': 1, 'hidden': True})
    worksheet.set_column(first_row['24фармацея'][0]-1, first_row['24фармацея'][0]-1, 7, table_format)
    worksheet.set_column(first_row['24фармацея'][0], first_row['24фармацея'][0], 7, percent_format)
    worksheet.set_column(first_row['24фармацея'][0]+1, first_row['24фармацея'][0]+1, 7, table_format)

    worksheet.set_column(first_row['Остатки'][1]+1, first_row['Остатки'][0] - 2, 7, table_format, {'level': 1, 'hidden': True})
    worksheet.set_column(first_row['Остатки'][0]-1, first_row['Остатки'][0]-1, 7, table_format)
    worksheet.set_column(first_row['Остатки'][0], first_row['Остатки'][0], 7, percent_format)
    worksheet.set_column(first_row['Остатки'][0]+1, first_row['Остатки'][0]+1, 7, table_format)
    

    writer.save()

In [52]:
def pivot(local_df, category, name, prod_or_ostat):
    category_df = local_df.loc[local_df['Категория'] == category].fillna('')
    categori_pivot = pd.pivot_table(category_df, values='Среднее', index=['Код аналога', 'Код ЛС', 'Код ЕФ',
    'Наименование ЛС', 'Вид ЛС', 'ФГ1', 'ФГ2', 'Rx/OTC', 'ЖНВЛП',
    'Маркетинговая категория'], columns=['Аптека'])

    col = categori_pivot.columns
    categori_pivot[f'Кол-во {prod_or_ostat}'] = categori_pivot.apply(lambda x: sum([1 for y in col if x[y] > 0]), axis=1)
    categori_pivot[f'Доля {prod_or_ostat}'] =  categori_pivot[f'Кол-во {prod_or_ostat}'] / len(col)
    categori_pivot[f'Среднее кол-во {prod_or_ostat}'] = categori_pivot[col].sum(axis=1) / categori_pivot[f'Кол-во {prod_or_ostat}']
    arrays = [[name] * len(categori_pivot.columns), categori_pivot.columns]
    categori_pivot.columns = pd.MultiIndex.from_arrays(arrays)
    return categori_pivot




for cat in prodazhi['Категория'].unique():    
   
    categori_zal_pivot = pivot(prodazhi, cat, 'зал', 'продаж')
    farm_pivot = pivot(farmacia, cat, '24фармация', 'продаж')
    aptekaru_pivot = pivot(aptekaru, cat, 'Аптека.ру', 'продаж')
    ostatki_cat_pivot = pivot(ostatki, cat, 'Остатки', 'остатков')

    all_pivot = categori_zal_pivot.join(farm_pivot, how='outer')
    all_pivot = all_pivot.join(aptekaru_pivot, how='outer')
    all_pivot = all_pivot.join(ostatki_cat_pivot, how='outer')

    d3d4_cat = d3d4.loc[d3d4['Категория'] == cat].fillna('')
    d3d4_pivot = pd.pivot_table(d3d4_cat, values='Группа товара', index=['Код ЛС'], columns=['Аптека'],  aggfunc=','.join)
    d3d4_col = d3d4_pivot.columns
    d3d4_pivot = d3d4_pivot.reset_index()
    d3d4_pivot['Кол-во аптек с Д3'] = d3d4_pivot.apply(lambda x: sum([1 for y in d3d4_pivot if x[y] == 'D3']), axis=1)
    d3d4_pivot['Кол-во аптек с Д4'] = d3d4_pivot.apply(lambda x: sum([1 for y in d3d4_pivot if x[y] == 'D4']), axis=1)
    d3d4_pivot['Доля аптек с Д3'] = d3d4_pivot['Кол-во аптек с Д3'] / len(d3d4_col)
    d3d4_pivot['Доля аптек с Д4'] = d3d4_pivot['Кол-во аптек с Д4'] / len(d3d4_col)
    arrays = [['код']+['Д3Д4']*len(d3d4_pivot.columns[1:]), d3d4_pivot.columns]
    d3d4_pivot.columns = pd.MultiIndex.from_arrays(arrays)

    all_pivot = all_pivot.reset_index()
    arrays = [['код'] * 3 + ['наименование'] + ['х-ки'] * 6 +[x[0] for x in all_pivot.columns][10:],
    [x[0] for x in all_pivot.columns][:10] + [x[1] for x in all_pivot.columns][10:]]
    all_pivot.columns = pd.MultiIndex.from_arrays(arrays)
    all_pivot = all_pivot.merge(d3d4_pivot, on = [all_pivot.columns[1]], how='left')


    all_pivot = all_pivot.merge(stop_list, on=[all_pivot.columns[1]], how='left')

    categorii_abc = abc.loc[abc['Категория'] == cat].fillna('')
    abc_pivot = pd.pivot_table(categorii_abc,  index=['Код ЛС'], columns=['Аптека'], values='Группа товара', aggfunc=','.join)
    abc_col = abc_pivot.columns
    abc_pivot = abc_pivot.reset_index()
    abc_pivot['Кол-во аптек с данной позицией в качестве ядра'] = abc_pivot.apply(lambda x: sum([1 for y in abc_pivot if x[y] in abc_list]), axis=1)
    abc_pivot['Доля аптек с данной позицией в качестве ядра во всей категории'] = abc_pivot['Кол-во аптек с данной позицией в качестве ядра'] / len(abc_col)
    arrays = [['код']+['ABC']*len(abc_pivot.columns[1:]), abc_pivot.columns]
    abc_pivot.columns = pd.MultiIndex.from_arrays(arrays)
    all_pivot = all_pivot.merge(abc_pivot, on = [all_pivot.columns[1]], how='left')


    all_pivot = all_pivot[['код','наименование', 'х-ки','ABC', 'Зал','Д3Д4', 'Апека.ру','24фармацея','Остатки']] #установка последовательности колонок
    # запоминаем индесы первой строки
    first_row =  {x[0]: [] for x in  all_pivot.columns}
    for x, y in enumerate(all_pivot.columns):
        first_row[y[0]].append(x)
    first_row = {name: (max(col), min(col) )for name, col  in first_row.items()}
    second_row = {y:x[1] for y, x in enumerate(all_pivot.columns)}
    get_excel(all_pivot,cat,first_row,second_row)

